# 0. Instructions and setup

## 0.1. Instructions. Part 2: Data Scientist Challenge (3.5 points)

- **Objective:** Explore different techniques to enhance model performance with limited  labeled data. You will be limited to 32 labeled examples in your task.  The rest can be viewed as unlabelled data. 

- **Tasks:**
  - **a. BERT Model with Limited Data (0.5 points):** Train a BERT-based model using only 32 labeled examples and assess its performance.
  - **b. Dataset Augmentation (1 point):** Experiment with an automated technique to increase your dataset size **without using LLMs** (chatGPT / Mistral / Gemini / etc...). Evaluate the impact on model performance.
  - **c. Zero-Shot Learning with LLM (0.5 points):** Apply a LLM (chatGPT/Claude/Mistral/Gemini/...) in a zero-shot learning setup. Document the performance.
  - **d. Data Generation with LLM (1 point):** Use a LLM (chatGPT/Claude/Mistral/Gemini/...) to generate new, labeled  dataset points. Train your BERT model with it + the 32 labels. Analyze  how this impacts model metrics.
  - **e. Optimal Technique Application (0.5 points):** Based on the previous experiments, apply the most effective  technique(s) to further improve your model's performance. Comment your results and propose improvements.

## 0.2. Libraries

In [1]:
# # !pip install polars  # Install polars for faster data processing

In [2]:
# Utilities
import numpy as np
import polars as pl
from library.metrics import Metrics
from library.utilities import set_seed
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import os
import matplotlib.pyplot as plt

# Deep Learning and NLP
import torch
import setfit
from transformers import (AutoTokenizer, AutoModel,
                          AutoModelForSequenceClassification,
                          TrainingArguments, Trainer,
                          EarlyStoppingCallback)

2025-06-12 22:50:41.984965: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 22:50:41.993689: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749761442.004552   70266 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749761442.007620   70266 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749761442.016628   70266 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
# Initialize the metrics object to save the results
metrics = Metrics()

In [4]:
# Check the availability of a GPU
print(torch.cuda.is_available())

True


## 0.3. Random Seed

In [5]:
# Set random seed for reproducibility
seed = 42
set_seed(42)

Seed set to 42. This ensures reproducibility of results across runs.


## 0.4. Loading the data: Swiss Judgement Prediction

Source: https://huggingface.co/datasets/rcds/swiss_judgment_prediction

In [6]:
# Load the cleaned Parquet file
df = pl.read_parquet('swiss_judgment_prediction_fr&it_clean.parquet')

# Display the loaded DataFrame
print("\nLoaded DataFrame shape:", df.shape)
print("\nLoaded DataFrame schema:")
print(df.schema)
print("\nFirst few rows of the loaded DataFrame:")
df.head()


Loaded DataFrame shape: (35386, 9)

Loaded DataFrame schema:
Schema({'id': Int32, 'year': Int32, 'text': String, 'labels': Int64, 'language': String, 'region': String, 'canton': String, 'legal area': String, 'split': String})

First few rows of the loaded DataFrame:


id,year,text,labels,language,region,canton,legal area,split
i32,i32,str,i64,str,str,str,str,str
22014,2011,"""Faits: A. Le 28 octobre 2002 à…",0,"""fr""","""Région lémanique""","""ge""","""civil law""","""train"""
11593,2007,"""Faits : Faits : A. Le 17 avril…",1,"""fr""","""Région lémanique""","""ge""","""penal law""","""train"""
26670,2013,"""Faits: A. Par jugement du 2 ma…",0,"""fr""","""Région lémanique""","""vd""","""penal law""","""train"""
5864,2004,"""Faits: Faits: A. N._, née en 1…",1,"""fr""","""Région lémanique""","""vd""","""insurance law""","""train"""
16122,2009,"""Faits: A. Y._ est propriétaire…",0,"""fr""","""Région lémanique""","""ge""","""public law""","""train"""


In [7]:
# Split the DataFrame into training, validation and test sets
train_df = df.filter(pl.col('split') == 'train')
val_df = df.filter(pl.col('split') == 'validation')
test_df = df.filter(pl.col('split') == 'test')

# Filter rows to keep only French language data
train_df_fr = train_df.filter(pl.col('language') == 'fr')
val_df_fr = val_df.filter(pl.col('language') == 'fr')
test_df_fr = test_df.filter(pl.col('language') == 'fr')

# Delete the original data to free up memory
del df

# 1. BERT Model with Limited Data

Outline of the intermediate tasks:

1. Preprocessing Pipeline
   - Lowercasing, punctuation stripping (or not, depending on BERT tokenizer).
   - Sentencepiece/BPE tokenization via the CamemBERT (for French) or UmBERTo (for Italian).
   - (Optional) language tags if you merge FR+IT in one model.
2. Hold-out Split. Since you only have 32 labels: use stratified k-fold CV (e.g. 8 × 4-fold) to get reliable estimates, or leave-one-out if you want maximum training data per fold.
3. BERT Model with Only 32 Examples
   - Model Choice: Pick a multilingual BERT (mBERT) or separate CamemBERT/UmBERTo checkpoint. Alternatives:
     - Multilingual/monolingual models:
       - One multilingual [`Sentence-Transformer` model](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#original-models), such as `paraphrase-multilingual-mpnet-base-v2` (best multilingual performer), `paraphrase-multilingual-MiniLM-L12-v2` (similar performer as the former, but much faster and smaller) or `distiluse-base-multilingual-cased-v1` (worst of the bunch).
       -  [BERT multilingual base model (cased)](https://huggingface.co/google-bert/bert-base-multilingual-cased). [Uncased model](https://huggingface.co/google-bert/bert-base-multilingual-uncased) also available. For an even smaller model, the distilled version can be used: [`distilbert-base-multilingual-cased`](https://huggingface.co/distilbert/distilbert-base-multilingual-cased).
       -  [CamemBERT 2.0](https://huggingface.co/almanach/camembertv2-base) and [CamemBERTav2](https://huggingface.co/almanach/camembertav2-base), models trained with French text and explained in the paper: [CamemBERT 2.0: A Smarter French Language Model Aged to Perfection](https://arxiv.org/html/2411.08868v1#S3). These models supposedly improve over the performance of the original [CamemBERT](https://huggingface.co/almanach/camembert-base) model, explained in "[CamemBERT: a Tasty French Language Model](https://arxiv.org/abs/1911.03894)".
       -  [FlauBERT](https://huggingface.co/docs/transformers/en/model_doc/flaubert), another model pre-trained on French text. Paper: "[FlauBERT: Unsupervised Language Model Pre-training for French](https://arxiv.org/abs/1912.05372)". 
       -  [BERT Base Italian Cased](https://huggingface.co/dbmdz/bert-base-italian-cased), [Uncased](https://huggingface.co/dbmdz/bert-base-italian-uncased), ,and [XXL Uncased](https://huggingface.co/dbmdz/bert-base-italian-xxl-uncased). 
       -  [UmBERTo Commoncrawl Cased](https://huggingface.co/Musixmatch/umberto-commoncrawl-cased-v1), another model trained with a large corpus of texts in Italian.
      -  Domain-specific models (law):
          - [LEGAL-BERT](https://huggingface.co/nlpaueb/legal-bert-base-uncased) does not seem to be a good option as it was trained only on English data.
          - [JuriBERT](https://huggingface.co/dascim/juribert-base) for legal texts in French. Paper explaining the model: [JuriBERT: A Masked-Language Model Adaptation for French Legal Text](https://arxiv.org/pdf/2110.01485).
          - [ITALIAN-LEGAL-BERT](https://huggingface.co/dlicari/Italian-Legal-BERT) for legal text in Italian. Paper explaining the model: [ITALIAN-LEGAL-BERT models for improving natural language processing tasks in the Italian legal domain](https://www.sciencedirect.com/science/article/pii/S0267364923001188).
     - Models to try: 
       - For both languages: `paraphrase-multilingual-MiniLM-L12-v2` (of the SentenceTransformers, good balance between performance and speed), BERT multilingual base model (cased).
       - For French: CamemBERT, FlauBERT, CamemBERTav2 (based on DebertaV2 architecture), JuriBERT.
       - For Italian: BERT Base Italian Cased, UmBERTo Commoncrawl Cased, ITALIAN-LEGAL-BERT.
     - [Model memory estimator](https://huggingface.co/docs/accelerate/en/usage_guides/model_size_estimator): in order to know whether a model actually fits in the memory of your computer.
   - Fine-tuning Setup.
     - Freeze or unfreeze last n encoder layers—try both.
     - Small learning rate (2e-5 – 5e-5), batch size = 8 or 16.
     - Early stopping on validation loss.
4. Training & Evaluation
    - Run your k-fold CV training loops.
    - Track accuracy, F1, precision, recall per fold.
    - Report mean ± std.
5. Error Analysis and feature interpretation.
    - Use `LIME` for analyzing the most relevant features for classifying the texts. 
    - Look at which examples are mispredicted.
    - Check language breakdown (FR vs. IT) to see if one is harder.

## 1.1. Using SetFit ("Sentence Transformer Fine-Tuning")

Notebook of reference: `Session_6_2_Zero_Shot_Classification.ipynb`, introduction, step 1 (loading data) and step 6 ("Few-Shot Classification with SetFit").

Applying SetFit (the “Sentence Transformer Fine-Tuning” recipe) can be regarded as **training** (it fine-tunes a pre-trained sentence-embedding model, plus fits a small classifier on top). Furthermore, SetFit was built **for** getting strong performance with as few as a few dozen labeled examples.

---

**Why SetFit can be regarded as training**

- **Contrastive fine-tuning:**
  We start with a frozen (or lightly unfrozen) Sentence-Transformer model and then *fine-tune* it on automatically generated sentence pairs derived from your 32 labels.
- **Classifier head training:**
  After contrastive tuning, SetFit fits a lightweight logistic-regression (or small MLP) classifier on the resulting embeddings.
- Both steps update model parameters—so it’s training/fine-tuning, not mere prompt-engineering or zero-shot.

---

**Why SetFit excels in limited-label regimes**

1. **Data amplification via contrastive pairs**

   - From each labeled example, SetFit creates positive pairs (e.g. two different augmentations of the same sentence) and negative pairs (across classes), turning 32 labels into hundreds or thousands of pairwise signals.
   - That extra signal helps the embedding space separate classes, even when you only have a few “gold” labels.

1. **Lightweight classifier**

   * Because the embedding model has already been tuned to distinguish the classes, the final classifier can be a simple logistic or MLP—so it needs very few examples to learn decision boundaries.

2. **Empirical few-shot strength**

   * In benchmarks, SetFit often outperforms standard BERT fine-tuning with few labels, and it’s much faster to train (no full back-prop through all BERT layers).

---

Note that there are 2 aspects that we need to implement for SetFit to work properly:

1. **Balance the examples of positive and negative classes** that will be passed for contrastive learning. Since approximately 29% of the labels are positive (approved motions) and the other 71% correspond to rejected motions, passing examples for contrastive learning without balancing the clases would bias the embedding space towards the majority class.
   
2. We need to **run several iterations** in order to get lower variability in the results, as a single training run can be highly variable due to randomness in the example sampling (as we only pass 32 labelled examples). Therefore, if we don't run several iterations for SetFit, the results would be highly noisy. Averaging across iterations smooths out this variance and gives us the true expected performance.

### Utilities

In [8]:
# Function to sample balanced dataset
def sample_balanced_dataset(dataset: pl.DataFrame, num_samples, seed):

    """
    Sample a balanced dataset with equal numbers of positive and negative examples.
    
    Args:
        dataset (pl.DataFrame): The input dataset containing 'text' and 'label' columns.
        num_samples (int): Total number of samples to return, must be even.
        seed (int): Random seed for reproducibility.
    Returns:
        Dataset: A balanced Dataset object with equal numbers of positive and negative examples.
    """

    # Get positive and negative examples
    pos_examples = dataset.filter(pl.col('label') == 1)
    neg_examples = dataset.filter(pl.col('label') == 0)
    
    # Sample equal numbers from each class
    samples_per_class = num_samples // 2
    
    if seed is not None:
        pos_sampled = pos_examples.sample(n=samples_per_class, shuffle=True, seed=seed)
        neg_sampled = neg_examples.sample(n=samples_per_class, shuffle=True, seed=seed)
    else:
        raise ValueError("Seed must be provided for reproducibility.")
    
    # Concatenate the sampled DataFrames, only the text and the label columns
    pos_sampled = pos_sampled.select(['text', 'label'])
    neg_sampled = neg_sampled.select(['text', 'label'])
    df = pl.concat([pos_sampled, neg_sampled], how='vertical')

    # Combine the datasets into a single Dataset object
    combined = Dataset.from_polars(df)
    
    return combined

In [11]:
# Function to run SetFit training and evaluation routine
def run_setfit_training(train_df: pl.DataFrame, val_df: pl.DataFrame, 
                        model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                        num_epochs=5, batch_size=16, learning_rate=2e-5, sample_size=32,
                        metric='f1', num_iterations=10, seed=42):
    """
    Run SetFit training and evaluation routine.
    
    Args:
        train_df (pl.DataFrame): Training DataFrame with 'text' and 'label' columns.
        val_df (pl.DataFrame): Validation DataFrame with 'text' and 'label' columns.
        model_name (str): Pretrained model name for SetFit.
        num_epochs (int): Number of epochs for training.
        batch_size (int): Batch size for training.
        learning_rate (float): Learning rate for the optimizer.
        sample_size (int): Number of samples to use for training in each iteration.
        metric (str): Metric to optimize during training ('f1', 'accuracy', etc.).
        num_iterations (int): Number of iterations to run the training process.
        seed (int): Random seed for reproducibility.
    
    Returns:
        list: A list of dictionaries containing evaluation metrics for each iteration.
    """
    
    # Prepare the validation set
    val_set = Dataset.from_polars(val_df.select(['text', 'label']))

    # Store results across iterations (for different metrics and iterations)
    iteration_results = []
    best_score = 0
    best_model = None
    best_iteration = 0

    # Run the sampling and training process with SetFit
    for iteration in range(num_iterations):
        print(f"\nIteration {iteration + 1}/{num_iterations}")

        # Create a fresh model for each iteration to avoid contamination
        model = setfit.SetFitModel.from_pretrained(model_name)

        # Sample balanced training data
        train_samples = sample_balanced_dataset(train_df, sample_size, seed + iteration)

        # Create the training arguments
        train_args = setfit.TrainingArguments(
            num_epochs=(num_epochs, num_epochs),  # Tuple format: (sentence_transformer_epochs, head_epochs)
            batch_size=(batch_size, batch_size),  # Tuple format: (sentence_transformer_batch, head_batch)
            body_learning_rate=(learning_rate, learning_rate),  # Tuple format for body learning rate (first for sentence transformer, second for head classifier)
        )

        # Initialize and train SetFit model
        trainer = setfit.Trainer(
            model=model,
            train_dataset=train_samples,  # Pairs of text and labels for Contrastive Learning
            eval_dataset=val_set,  # Validation set for evaluation
            metric=metric,  # Metric to optimize
            args=train_args  # Training arguments
        )

        trainer.train()  # Train the model
        print("Training completed.")

        # Evaluate on validation set (for hyperparameter tuning/model selection)
        val_predictions = trainer.model.predict(val_set['text'])
        val_metrics = {
            'accuracy': accuracy_score(val_set['label'], val_predictions),
            'f1': f1_score(val_set['label'], val_predictions),
            'precision': precision_score(val_set['label'], val_predictions),
            'recall': recall_score(val_set['label'], val_predictions)
        }
        
        # Store results for this iteration
        iteration_results.append(val_metrics)
        print(f"Validation F1: {val_metrics['f1']:.4f}")

        # Check if this is the best model so far
        current_score = val_metrics[metric]
        if current_score > best_score:
            best_score = current_score
            best_iteration = iteration + 1
            # Clean up previous best model
            if best_model is not None:
                del best_model
            # Store reference to current best model
            best_model = trainer.model
            print(f"New best model found with {metric}: {best_score:.4f}")

        # Clean up memory for this iteration - SINGLE, CLEAR LOGIC
        if best_model is trainer.model:
            # This is the best model, only delete trainer
            del model, trainer
        else:
            # This is not the best model, delete everything
            del model, trainer

        # Clear CUDA cache if using GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()  # Wait for all operations to complete
    
    print('Finished training in all iterations. Saving the results to a parquet file...')
    
    # Save the best model
    san_model_name = model_name.split(sep='/')[-1]  # Sanitize model name for file path, keep only the last part
    model_path = os.path.join('models', 'part_2', 'a', f'setfit_best_{san_model_name}')
    os.makedirs(model_path, exist_ok=True)
    best_model.save_pretrained(model_path)  # Save the best model to the specified path
    print(f'Best model saved to: {model_path}')

    # Save the results in a polars DataFrame
    results_df = pl.DataFrame(iteration_results)
    results_df = results_df.with_columns(pl.Series("iteration", range(1, len(results_df) + 1)))  # Add iteration number to the results DataFrame

    # Save the results DataFrame to a Parquet file
    results_path = os.path.join('results', 'part_2', 'a', f'setfit_results_{san_model_name}.parquet')
    os.makedirs(os.path.dirname(results_path), exist_ok=True)  # Ensure the directory exists
    results_df.write_parquet(results_path)
    print(f'Results saved to: {results_path}')

    return results_df

Note that SetFit only natively supports sentence-transformer models: passing one which is [not](https://huggingface.co/models?library=sentence-transformers&author=sentence-transformers) (see also the [docs](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#original-models)) will make SetFit automatically wrap the BERT model with a sentence-transformers layer using mean pooling. Using a model that is not part of the `sentence-transformers` will yield the following message:

> No sentence-transformers model found with name google-bert/bert-base-multilingual-cased. Creating a new one with mean pooling.

However, for better performance with SetFit, it is preferred to use a model that was specifically pre-trained as a sentence transformer.

   - Model Choice: Pick a multilingual BERT (mBERT) or separate CamemBERT/UmBERTo checkpoint. Alternatives:
     - Multilingual/monolingual models:
       - One multilingual [`Sentence-Transformer` model](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#original-models), such as `paraphrase-multilingual-mpnet-base-v2` (best multilingual performer), `paraphrase-multilingual-MiniLM-L12-v2` (similar performer as the former, but much faster and smaller) or `distiluse-base-multilingual-cased-v1` (worst of the bunch).
       -  [BERT multilingual base model (cased)](https://huggingface.co/google-bert/bert-base-multilingual-cased). [Uncased model](https://huggingface.co/google-bert/bert-base-multilingual-uncased) also available. Paper: "[BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)". 
       -  [CamemBERT 2.0](https://huggingface.co/almanach/camembertv2-base) and [CamemBERTav2](https://huggingface.co/almanach/camembertav2-base), models trained with French text and explained in the paper: [CamemBERT 2.0: A Smarter French Language Model Aged to Perfection](https://arxiv.org/html/2411.08868v1#S3). These models supposedly improve over the performance of the original [CamemBERT](https://huggingface.co/docs/transformers/en/model_doc/camembert) model, explained in "[CamemBERT: a Tasty French Language Model](https://arxiv.org/abs/1911.03894)".
       -  [FlauBERT](https://huggingface.co/docs/transformers/en/model_doc/flaubert), another model pre-trained on French text. Paper: "[FlauBERT: Unsupervised Language Model Pre-training for French](https://arxiv.org/abs/1912.05372)". 
       -  [BERT Base Italian Cased](https://huggingface.co/dbmdz/bert-base-italian-cased), [Uncased](https://huggingface.co/dbmdz/bert-base-italian-uncased), ,and [XXL Uncased](https://huggingface.co/dbmdz/bert-base-italian-xxl-uncased). 
       -  [UmBERTo Commoncrawl Cased](https://huggingface.co/Musixmatch/umberto-commoncrawl-cased-v1), another model trained with a large corpus of texts in Italian.
      -  Domain-specific models (law):
          - [LEGAL-BERT](https://huggingface.co/nlpaueb/legal-bert-base-uncased) does not seem to be a good option as it was trained only on English data.
          - [JuriBERT](https://huggingface.co/dascim/juribert-base) for legal texts in French. Paper explaining the model: [JuriBERT: A Masked-Language Model Adaptation for French Legal Text](https://arxiv.org/pdf/2110.01485).
          - [ITALIAN-LEGAL-BERT](https://huggingface.co/dlicari/Italian-Legal-BERT) for legal text in Italian. Paper explaining the model: [ITALIAN-LEGAL-BERT models for improving natural language processing tasks in the Italian legal domain](https://www.sciencedirect.com/science/article/pii/S0267364923001188).

Models to try: 
- For both languages: `paraphrase-multilingual-MiniLM-L12-v2` (of the SentenceTransformers, good balance between performance and speed), distilled BERT multilingual base model (cased) ([`distilbert-base-multilingual-cased`](https://huggingface.co/distilbert/distilbert-base-multilingual-cased).)
- For French: CamemBERTav2 (based on DebertaV2 architecture), JuriBERT.
- For Italian: BERT Base Italian Cased, ITALIAN-LEGAL-BERT.

[Model memory estimator](https://huggingface.co/docs/accelerate/en/usage_guides/model_size_estimator): in order to know whether a model actually fits in the memory of your computer:



Finally, consider that there might be some memory issues by running SetFit:  ( https://github.com/huggingface/setfit/issues/472 ). Adapt the hyperparameters consequently to your memory limitations.

### Multilingual models

#### `paraphrase-multilingual-MiniLM-L12-v2`

In [ ]:
# Rename columns to match SetFit requirements (labels should be 'labels'), creating a copy
train_df_setfit = train_df.clone()
val_df_setfit = val_df.clone()
train_df_setfit = train_df_setfit.rename({'labels': 'label'})
val_df_setfit = val_df_setfit.rename({'labels': 'label'})

# Define sample size (labelled examples for training)
sample_size = 32
num_iterations = 10  # Run several iterations for the sample size (32 labels) to minimize the impact of randomness
metric = 'f1'  # Metric to optimize
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # Path to the pre-trained model
batch_size = 64 # Batch size for training (reduce if you run into memory issues, but larger batch sizes will speed up training)
num_epochs = 5  # Number of epochs for training
learning_rate= 2e-5  # Learning rate for the optimizer

# Create a subset of the validation data frame for faster processing
sample_size_val = 500
val_df_setfit = val_df_setfit.sample(n=sample_size_val, shuffle=True, seed=seed)

results_m1 = run_setfit_training(
    train_df=train_df_setfit, 
    val_df=val_df_setfit, 
    model_name=model_name, 
    num_epochs=num_epochs, 
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    sample_size=sample_size, 
    metric=metric, 
    num_iterations=num_iterations, 
    seed=seed
)

In [13]:
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'

results_m1 = pl.read_parquet(os.path.join('results', 'part_2', 'a', f'setfit_results_{model_name}.parquet'))

print(f'Results for model: {model_name}')
display(results_m1)

# Print mean of all metrics across iterations
metrics = ['accuracy', 'f1', 'precision', 'recall']
mean_metrics = results_m1.select([pl.col(metric).mean().alias(metric) for metric in metrics]).to_dict(as_series=False)
print("\nMean metrics across all iterations:")
print(mean_metrics)

Results for model: paraphrase-multilingual-MiniLM-L12-v2


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.55,0.285714,0.204545,0.473684,1
0.546,0.250825,0.182692,0.4,2
0.49,0.25656,0.177419,0.463158,3
0.52,0.259259,0.183406,0.442105,4
0.458,0.306905,0.202703,0.631579,5
0.502,0.233846,0.165217,0.4,6
0.544,0.283019,0.201794,0.473684,7
0.436,0.26178,0.174216,0.526316,8
0.384,0.283721,0.18209,0.642105,9



Mean metrics across all iterations:
{'accuracy': [0.5010000000000001], 'f1': [0.26495709982755045], 'precision': [0.1849223869644958], 'recall': [0.47789473684210526]}


#### BERT-base multilingual cased

In [ ]:
# Rename columns to match SetFit requirements (labels should be 'labels'), creating a copy
train_df_setfit = train_df.clone()
val_df_setfit = val_df.clone()
train_df_setfit = train_df_setfit.rename({'labels': 'label'})
val_df_setfit = val_df_setfit.rename({'labels': 'label'})

# Define sample size (labelled examples for training)
sample_size = 32
num_iterations = 5  # Run several iterations for the sample size (32 labels) to minimize the impact of randomness
metric = 'f1'  # Metric to optimize
model_name = "google-bert/bert-base-multilingual-cased"  # Path to the pre-trained model
batch_size = 8 # Batch size for training (reduce if you run into memory issues, but larger batch sizes will speed up training)
num_epochs = 5  # Number of epochs for training
learning_rate= 2e-5  # Learning rate for the optimizer

# Create a subset of the validation data frame for faster processing
sample_size_val = 500
val_df_setfit = val_df_setfit.sample(n=sample_size_val, shuffle=True, seed=seed)

results_m2 = run_setfit_training(
    train_df=train_df_setfit, 
    val_df=val_df_setfit, 
    model_name=model_name, 
    num_epochs=num_epochs, 
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    sample_size=sample_size, 
    metric=metric, 
    num_iterations=num_iterations, 
    seed=seed
)

In [ ]:
model_name = 'bert-base-multilingual-cased'

results_m2 = pl.read_parquet(os.path.join('results', 'part_2', 'a', f'setfit_results_{model_name}.parquet'))

print(f'Results for model: {model_name}')
display(results_m2)

# Print mean of all metrics across iterations
metrics = ['accuracy', 'f1', 'precision', 'recall']
mean_metrics = results_m2.select([pl.col(metric).mean().alias(metric) for metric in metrics]).to_dict(as_series=False)
print("\nMean metrics across all iterations:")
print(mean_metrics)

Results for model: google-bert/bert-base-multilingual-cased


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.532,0.26875,0.191111,0.452632,1
0.42,0.299517,0.194357,0.652632,2
0.61,0.229249,0.183544,0.305263,3
0.464,0.298429,0.198606,0.6,4
0.496,0.296089,0.201521,0.557895,5


### Models pre-trained in French

#### CamemBERT

In [ ]:
# Rename columns to match SetFit requirements (labels should be 'labels'), creating a copy
train_fr_df = train_df.clone()
val_fr_df = val_df.clone()
# Keep only the columns where the language is French
train_fr_df = train_fr_df.filter(pl.col('language') == 'fr').rename({'labels': 'label'})
val_fr_df = val_fr_df.filter(pl.col('language') == 'fr').rename({'labels': 'label'})

# Define sample size (labelled examples for training)
sample_size = 32
num_iterations = 5  # Run several iterations for the sample size (32 labels) to minimize the impact of randomness
metric = 'f1'  # Metric to optimize
model_name = "almanach/camembert-base"  # Path to the pre-trained model
batch_size = 8  # Batch size for training (reduce if you run into memory issues, but larger batch sizes will speed up training)
num_epochs = 5  # Number of epochs for training
learning_rate= 2e-5  # Learning rate for the optimizer

# Create a subset of the validation data frame for faster processing
sample_size_val = 500
val_fr_df = val_fr_df.sample(n=sample_size_val, shuffle=True, seed=seed)

results_m3 = run_setfit_training(
    train_df=train_fr_df, 
    val_df=val_fr_df, 
    model_name=model_name, 
    num_epochs=num_epochs, 
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    sample_size=sample_size, 
    metric=metric, 
    num_iterations=num_iterations, 
    seed=seed
)

In [26]:
model_name = 'camembert-base'

results_m3 = pl.read_parquet(os.path.join('results', 'part_2', 'a', f'setfit_results_{model_name}.parquet'))

print(f'Results for model: {model_name}')
display(results_m3)

# Print the metrics for the epoch with the lowest validation loss
best_epoch = results_m3.filter(pl.col('f1') == results_m3['f1'].max())
print("\nBest iteration metrics (highest F1-score):")
display(best_epoch)

Results for model: camembert-base


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.642,0.286853,0.25,0.336449,1
0.644,0.245763,0.224806,0.271028,2
0.446,0.31941,0.216667,0.607477,3
0.46,0.341463,0.231023,0.654206,4
0.628,0.243902,0.215827,0.280374,5



Best iteration metrics (highest F1-score):


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.46,0.341463,0.231023,0.654206,4


#### JuriBERT-base

In [ ]:
# Rename columns to match SetFit requirements (labels should be 'labels'), creating a copy
train_fr_df = train_df.clone()
val_fr_df = val_df.clone()
# Keep only the columns where the language is French
train_fr_df = train_fr_df.filter(pl.col('language') == 'fr').rename({'labels': 'label'})
val_fr_df = val_fr_df.filter(pl.col('language') == 'fr').rename({'labels': 'label'})

# Define sample size (labelled examples for training)
sample_size = 32
num_iterations = 5  # Run several iterations for the sample size (32 labels) to minimize the impact of randomness
metric = 'f1'  # Metric to optimize
model_name = "dascim/juribert-base"  # Path to the pre-trained model
batch_size = 8  # Batch size for training (reduce if you run into memory issues, but larger batch sizes will speed up training)
num_epochs = 5  # Number of epochs for training
learning_rate= 2e-5  # Learning rate for the optimizer

# Create a subset of the validation data frame for faster processing
sample_size_val = 500
val_fr_df = val_fr_df.sample(n=sample_size_val, shuffle=True, seed=seed)

results_m4 = run_setfit_training(
    train_df=train_fr_df, 
    val_df=val_fr_df, 
    model_name=model_name, 
    num_epochs=num_epochs, 
    batch_size=batch_size, 
    learning_rate=learning_rate, 
    sample_size=sample_size, 
    metric=metric, 
    num_iterations=num_iterations, 
    seed=seed
)

In [25]:
model_name = 'juribert-base'

results_m4 = pl.read_parquet(os.path.join('results', 'part_2', 'a', f'setfit_results_{model_name}.parquet'))

print(f'Results for model: {model_name}')
display(results_m4)

# Print the metrics for the epoch with the lowest validation loss
best_epoch = results_m4.filter(pl.col('f1') == results_m4['f1'].max())
print("\nBest iteration metrics (highest F1-score):")
display(best_epoch)

Results for model: juribert-base


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.606,0.318339,0.252747,0.429907,1
0.418,0.285012,0.193333,0.542056,2
0.522,0.307246,0.222689,0.495327,3
0.578,0.259649,0.207865,0.345794,4
0.574,0.292359,0.226804,0.411215,5



Best iteration metrics (highest F1-score):


accuracy,f1,precision,recall,iteration
f64,f64,f64,f64,i64
0.606,0.318339,0.252747,0.429907,1


## 1.2. Standard fine-tuning

Notebook of reference: `Session_5_1_BERT_HF_Implementation.ipynb`, sections 1, 2 and 5.

### With CamemBERT

First, we set basic parameters that we will need for fine-tuning a model based on the BERT architecture.

In [9]:
model_name = "almanach/camembert-base"  # Path to the pre-trained model
num_labels = 2  # Number of labels for the classification task (in this case, binary classification)
max_length = min(int(AutoModel.from_pretrained(model_name).config.max_position_embeddings), 512)  # Maximum length of the input sequences (truncation if larger than this). Set dynamically based on the chosen model.

print(f"Model: {model_name}, Max Length: {max_length}")

Model: almanach/camembert-base, Max Length: 512


Second, we tokenize the text and create the vocabulary IDs in order to create the inputs for the BERT-based model.

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
sample_text = train_df.select('text')['text'][0]  # Select the text of the first sample

# Tokenize the sample text considering the maximum length
tokens = tokenizer(
    sample_text, 
    max_length=max_length,  # Truncate to the maximum length of the model 
    truncation=True, 
    padding='max_length'  # Pad to the maximum length
)

# Display input IDs (tokenized & padded)
print(f"Sample text: {sample_text}")
print("Token IDs:", tokens["input_ids"])
print("Length:", len(tokens["input_ids"]))
print("Attention Mask:", tokens["attention_mask"])

Sample text: Faits: A. Le 28 octobre 2002 à 14 h.50, A._, domiciliée en France, circulait, au volant d'une voiture dont elle est détentrice, sur la route d'Hermance (GE) en direction de Corsier, après avoir quitté une place de stationnement. Alors qu'elle obliquait à gauche pour entrer dans le parking de la poste de Collonge-Bellerive en vue d'effectuer un demi-tour, l'arrière gauche de sa voiture a été heurté par un scooter de marque Honda Fes piloté par son détenteur, B._. Ce dernier a été grièvement blessé. Au lieu de l'accident, la vitesse maximale autorisée était de 50 km/h. A l'endroit où l'automobiliste a obliqué à gauche - manoeuvre qui était permise -, la ligne centrale est pointillée, alors qu'elle est pleine avant et après cet endroit. Le scooter a laissé une trace de freinage de 9,7 mètres qui commence sur la ligne de sécurité et évolue vers la gauche. La conductrice de la voiture a reconnu qu'elle n'avait pas regardé derrière elle avant d'obliquer. La police a infligé une 

In [12]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_length)

# We convert the Polars Data Frame to an arrow Dataset and get a sample of the training data
if 'labels' in train_df_fr.columns:
    train_df_fr = train_df_fr.rename({'labels': 'label'})  # Rename 'labels' to 'label' for compatibility with sampling function
train_set_fr = sample_balanced_dataset(train_df_fr, 32, seed)
train_set_fr = train_set_fr.rename_column('label', 'labels')  # Rename the label column again to 'labels' for compatibility with Hugging Face Trainer
# train_set_fr = Dataset.from_polars(train_df_fr.select(['text', 'labels']).sample(n=32, shuffle=True, seed=seed))
val_set_fr = Dataset.from_polars(val_df_fr.select(['text', 'labels']))

# Now, we actually tokenize the datasets
train_set_fr = train_set_fr.map(tokenize, batched=True, remove_columns=["text"])
val_set_fr = val_set_fr.map(tokenize, batched=True, remove_columns=["text"])

# Convert the data in the datasets to PyTorch tensors
train_set_fr.set_format(type='torch')
val_set_fr.set_format(type='torch')

# Display the first example from the training set
print("\nFirst example from the training set:")
print(train_set_fr[0])

/home/pablo/.cache/pypoetry/virtualenvs/bse-nlp-YMIcxN07-py3.12/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/3095 [00:00<?, ? examples/s]


First example from the training set:
{'labels': tensor(1), 'input_ids': tensor([    5,  9447,    10,    43,  9447,    10,    43,   114,     9,   502,
          124,   108,   732,  7416,    36,   181,   667,  9474,   275,     9,
          862,    33, 11125,    20, 19377,     8,  4103,    31,   111,    96,
          101, 20796,    37,    13, 12088,     8,  4103,    25,  8259, 12254,
          537,  2143,   156,  9311,    38,   244,    26,   856,    92,    13,
         5454,     8,  4103,    53,    42,   340,  5981,  9637,    36,   205,
          667,  7416,     9,    54,   409,   652,  5985,     7,    51,   171,
         7145,    28,   400,     8,  2734,   843,     8,    17,    11,  8321,
        27333,     8,    17,    11,  2049,    26, 21556,    38,   244,    26,
          856,    92,    17,    11,  7720,    21,  7771,   120,   211,    23,
          327,     8,  1141,    25,  1157,   732,  9474,  4109,   666,    29,
           28,  1141,    25,   135,   871, 10801,    17,    11,  7720

The transformation has been executed as expected: we now have all of the features (input IDs) and labels saved as tensors. Remember that the attention mask explicits which tokens are actually real (1) and which are padded (0). 

Third, we initialize the `AutoModelForSequenceClassification` class from Hugging Face, which automatically handles the creation of a classification layer on top of the embedding layers of the BERT-based model. 

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=model_name,
    num_labels=num_labels,
    problem_type="single_label_classification"
)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at almanach/camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Fourth, we configure the relevant training arguments.

In [14]:
san_model_name = model_name.split(sep='/')[-1]  # Sanitize model name for file path, keep only the last part

train_args = TrainingArguments(
    output_dir = os.path.join('models', 'part_2', 'a', f'cls_fine_tuning_{san_model_name}'),
    eval_strategy = "epoch",  # Evaluate at the end of each epoch
    save_strategy = "epoch",  # Save the model at the end of each epoch
    logging_strategy = "steps",  # Log training progress every logging_steps
    logging_steps = 50,
    learning_rate = 2e-5,  # Learning rate for fine-tuning the language model
    per_device_train_batch_size = 16,  # Batch size for training
    per_device_eval_batch_size  = 16,  # Batch size for evaluation
    num_train_epochs = 20,  # Maximum number of epochs for training (unless early stopping is triggered)
    weight_decay = 0.01,  # Weight decay for regularization
    load_best_model_at_end = True,  # Load the best model at the end of training based on evaluation loss
    metric_for_best_model = "eval_loss",  # Metric to use for determining the best model
    save_total_limit = 2,  # Limit the total number of saved models to the best 2 models
    seed = seed,  # Set random seed for reproducibility
    report_to = "none",  # Disable wandb
    fp16 = True, # Enables mixed precision training (compatible on some GPUs, but not all). Set to True if you have a compatible GPU.
    gradient_accumulation_steps = 2  # Gradient accumulation steps to effectively increase the batch size without increasing memory usage
)

print("Output directory for the model:", train_args.output_dir)

Output directory for the model: models/part_2/a/cls_fine_tuning_camembert-base


Finally, we initialize the `Trainer` class, fine-tune the embedding model and train the classifier.

In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_set_fr,
    eval_dataset=val_set_fr.shuffle(seed=seed),  # Shuffle the validation set for evaluation
    compute_metrics=lambda p: {  # Return a dictionary of metrics
        "accuracy": accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),  # Compare the true labels with the predicted labels, by selecting the index with the highest probability
        "f1": f1_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        "precision": precision_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        "recall": recall_score(p.label_ids, np.argmax(p.predictions, axis=1))
    },
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # Early stopping after 3 epochs without improvement
)

train_output = trainer.train()

# Filter for epoch logs (those that have 'epoch' key)
log_history = trainer.state.log_history
epoch_logs = [log for log in log_history if 'epoch' in log]

# Convert to Polars DataFrame
results_df = pl.DataFrame(epoch_logs)

# Save results in a Parquet file
results_path = os.path.join('results', 'part_2', 'a', f'cls_fine_tuning_results_{san_model_name}.parquet')
os.makedirs(os.path.dirname(results_path), exist_ok=True)  # Ensure the directory exists
results_df.write_parquet(results_path)
print(f'Results saved to: {results_path}')

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.697425,0.276575,0.339333,0.209854,0.885978
2,No log,0.696253,0.335703,0.332901,0.210851,0.790447
3,No log,0.698141,0.267528,0.341563,0.210451,0.906009
4,No log,0.698896,0.256866,0.340218,0.209024,0.913713
5,No log,0.697306,0.338934,0.336576,0.213142,0.799692
6,No log,0.694188,0.490792,0.306948,0.214769,0.537750
7,No log,0.690351,0.621002,0.262728,0.221868,0.322034
8,No log,0.688712,0.649435,0.252240,0.228180,0.281972
9,No log,0.687847,0.655574,0.242898,0.225296,0.263482
10,No log,0.686955,0.660420,0.240058,0.226158,0.255778


In [18]:
# Load the results DataFrame from the Parquet file
results_path = os.path.join('results', 'part_2', 'a', f'cls_fine_tuning_results_camembert-base.parquet')
results_df = pl.read_parquet(results_path)

# Display the metrics DataFrame
print("\nMetrics DataFrame:")
display(results_df)

# Print the metrics for the epoch with the lowest validation loss
best_epoch = results_df.filter(pl.col('eval_loss') == results_df['eval_loss'].min()).select(['epoch', 'eval_loss', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall'])
print("\nBest epoch metrics (lowest validation loss):")
display(best_epoch)


Metrics DataFrame:


eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64
0.697425,0.276575,0.339333,0.209854,0.885978,17.2477,179.444,11.248,1.0,1,null,null,null,null,null
0.696253,0.335703,0.332901,0.210851,0.790447,17.599,175.862,11.023,2.0,2,null,null,null,null,null
0.698141,0.267528,0.341563,0.210451,0.906009,17.7029,174.83,10.959,3.0,3,null,null,null,null,null
0.698896,0.256866,0.340218,0.209024,0.913713,17.64,175.454,10.998,4.0,4,null,null,null,null,null
0.697306,0.338934,0.336576,0.213142,0.799692,17.6216,175.637,11.009,5.0,5,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
0.688309,0.634895,0.267185,0.230683,0.317411,17.5951,175.901,11.026,12.0,12,null,null,null,null,null
0.689218,0.617447,0.279805,0.231156,0.354391,17.6085,175.767,11.017,13.0,13,null,null,null,null,null
0.689515,0.607108,0.282172,0.228708,0.368259,17.6307,175.547,11.004,14.0,14,null,null,null,null,null



Best epoch metrics (lowest validation loss):


epoch,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall
f64,f64,f64,f64,f64,f64
10.0,0.686955,0.66042,0.240058,0.226158,0.255778


### With JuriBERT-base

In [13]:
model_name = "dascim/juribert-base"  # Path to the pre-trained model
num_labels = 2  # Number of labels for the classification task (in this case, binary classification)
max_length = min(int(AutoModel.from_pretrained(model_name).config.max_position_embeddings), 512)  # Maximum length of the input sequences (truncation if larger than this). Set dynamically based on the chosen model.

print(f"Model: {model_name}, Max Length: {max_length}")

Some weights of BertModel were not initialized from the model checkpoint at dascim/juribert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: dascim/juribert-base, Max Length: 512


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=max_length)

# We convert the Polars Data Frame to an arrow Dataset and get a sample of the training data
if 'labels' in train_df_fr.columns:
    train_df_fr = train_df_fr.rename({'labels': 'label'})  # Rename 'labels' to 'label' for compatibility with sampling function
train_set_fr = sample_balanced_dataset(train_df_fr, 32, seed)
train_set_fr = train_set_fr.rename_column('label', 'labels')  # Rename the label column again to 'labels' for compatibility with Hugging Face Trainer
# train_set_fr = Dataset.from_polars(train_df_fr.select(['text', 'labels']).sample(n=32, shuffle=True, seed=seed))
val_set_fr = Dataset.from_polars(val_df_fr.select(['text', 'labels']))

# Now, we actually tokenize the datasets
train_set_fr = train_set_fr.map(tokenize, batched=True, remove_columns=["text"])
val_set_fr = val_set_fr.map(tokenize, batched=True, remove_columns=["text"])

# Convert the data in the datasets to PyTorch tensors
train_set_fr.set_format(type='torch')
val_set_fr.set_format(type='torch')

# Display the first example from the training set
print("\nFirst example from the training set:")
print(train_set_fr[0])

# Load the model for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=model_name,
    num_labels=num_labels,
    problem_type="single_label_classification"
)

# Set training arguments
san_model_name = model_name.split(sep='/')[-1]  # Sanitize model name for file path, keep only the last part

train_args = TrainingArguments(
    output_dir = os.path.join('models', 'part_2', 'a', f'cls_fine_tuning_{san_model_name}'),
    eval_strategy = "epoch",  # Evaluate at the end of each epoch
    save_strategy = "epoch",  # Save the model at the end of each epoch
    logging_strategy = "steps",  # Log training progress every logging_steps
    logging_steps = 50,
    learning_rate = 2e-5,  # Learning rate for fine-tuning the language model
    per_device_train_batch_size = 16,  # Batch size for training
    per_device_eval_batch_size  = 16,  # Batch size for evaluation
    num_train_epochs = 20,  # Maximum number of epochs for training (unless early stopping is triggered)
    weight_decay = 0.01,  # Weight decay for regularization
    load_best_model_at_end = True,  # Load the best model at the end of training based on evaluation loss
    metric_for_best_model = "eval_loss",  # Metric to use for determining the best model
    save_total_limit = 2,  # Limit the total number of saved models to the best 2 models
    seed = seed,  # Set random seed for reproducibility
    report_to = "none",  # Disable wandb
    fp16 = True, # Enables mixed precision training (compatible on some GPUs, but not all). Set to True if you have a compatible GPU.
    gradient_accumulation_steps = 2  # Gradient accumulation steps to effectively increase the batch size without increasing memory usage
)

print("Output directory for the model:", train_args.output_dir)

/home/pablo/.cache/pypoetry/virtualenvs/bse-nlp-YMIcxN07-py3.12/lib/python3.12/site-packages/datasets/utils/_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/3095 [00:00<?, ? examples/s]


First example from the training set:
{'labels': tensor(1), 'input_ids': tensor([    0,    42,   322,    87,   548,   225,    42,   322,    87,   548,
          225,    37,    18,   225,    40,    89,   349,   279,   866,  2315,
          315,   627,   837,  1823,    16,   225,    52,    18,    67,   270,
         4158,   321,  2647,   265,  5875,   397,   596,   672,   438,  2312,
          308,   277,   225,    39,  1188,   265,  5875,   300,  1850,  7193,
         2680,  3163,   225,    55,    45,    38,   464,   311,    17,  3492,
           30,   277,  1263,   265,  5875,    13,   628,   935,  9366,   315,
          697,   837,  2315,    18,   225,    48,    73,  1076,   911,  2599,
           16,   425,   557,  2926,   402,   552,   265,  2308,  1852,   265,
          263,    11,    51,  2196, 20005,   325,   265,   263,    11,  1844,
           17,  6574,   464,   311,    17,  3492,    30,   263,    11,  2337,
          225,    37,    45,   652,   225,    40,   359,   342,  2170

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dascim/juribert-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Output directory for the model: models/part_2/a/cls_fine_tuning_juribert-base


In [ ]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=train_set_fr,
    eval_dataset=val_set_fr.shuffle(seed=seed),  # Shuffle the validation set for evaluation
    compute_metrics=lambda p: {  # Return a dictionary of metrics
        "accuracy": accuracy_score(p.label_ids, np.argmax(p.predictions, axis=1)),  # Compare the true labels with the predicted labels, by selecting the index with the highest probability
        "f1": f1_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        "precision": precision_score(p.label_ids, np.argmax(p.predictions, axis=1)),
        "recall": recall_score(p.label_ids, np.argmax(p.predictions, axis=1))
    },
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]  # Early stopping after 3 epochs without improvement
)

train_output = trainer.train()

# Filter for epoch logs (those that have 'epoch' key)
log_history = trainer.state.log_history
epoch_logs = [log for log in log_history if 'epoch' in log]

# Convert to Polars DataFrame
results_df = pl.DataFrame(epoch_logs)

# Save results in a Parquet file
results_path = os.path.join('results', 'part_2', 'a', f'cls_fine_tuning_results_{san_model_name}.parquet')
os.makedirs(os.path.dirname(results_path), exist_ok=True)  # Ensure the directory exists
results_df.write_parquet(results_path)
print(f'Results saved to: {results_path}')

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.675423,0.608724,0.253851,0.211499,0.317411
2,No log,0.644028,0.689499,0.215510,0.229167,0.203390
3,No log,0.639690,0.685299,0.220800,0.229617,0.212635
4,No log,0.645959,0.673667,0.237160,0.232593,0.241911


KeyboardInterrupt: 

In [19]:
# Load the results DataFrame from the Parquet file
results_path = os.path.join('results', 'part_2', 'a', f'cls_fine_tuning_results_juribert-base.parquet')
results_df = pl.read_parquet(results_path)

# Display the metrics DataFrame
print("\nMetrics DataFrame:")
display(results_df)

# Print the metrics for the epoch with the lowest validation loss
best_epoch = results_df.filter(pl.col('eval_loss') == results_df['eval_loss'].min()).select(['epoch', 'eval_loss', 'eval_accuracy', 'eval_f1', 'eval_precision', 'eval_recall'])
print("\nBest epoch metrics (lowest validation loss):")
display(best_epoch)


Metrics DataFrame:


eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch,step,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64
0.793118,0.231018,0.344714,0.209856,0.964561,17.0745,181.265,11.362,1.0,1,null,null,null,null,null
0.683671,0.564459,0.263388,0.204064,0.371341,17.4299,177.568,11.13,2.0,2,null,null,null,null,null
0.641419,0.71664,0.125623,0.177966,0.097072,17.6116,175.736,11.015,3.0,3,null,null,null,null,null
0.632946,0.719548,0.143984,0.2,0.112481,17.6293,175.56,11.004,4.0,4,null,null,null,null,null
0.640205,0.685299,0.193709,0.209302,0.180277,17.5936,175.916,11.027,5.0,5,null,null,null,null,null
0.657755,0.632633,0.239465,0.211584,0.275809,17.5726,176.126,11.04,6.0,6,null,null,null,null,null
0.681419,0.580614,0.277283,0.217088,0.383667,17.6266,175.587,11.006,7.0,7,null,null,null,null,null
0.703745,0.536349,0.309764,0.225175,0.496148,17.5646,176.207,11.045,8.0,8,null,null,null,null,null
0.722242,0.504685,0.318364,0.22375,0.551618,17.5546,176.307,11.051,9.0,9,null,null,null,null,null



Best epoch metrics (lowest validation loss):


epoch,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall
f64,f64,f64,f64,f64,f64
4.0,0.632946,0.719548,0.143984,0.2,0.112481


# 2. Dataset Augmentation

Outline of the intermediate tasks: We want a fully automated pipeline. A good candidate is Easy Data Augmentation (EDA) or back-translation via open‐source MT models.
1. Choose Technique(s)
   - Easy Data Augmentation techniques (random insertion, deletion, swap).
   - Random synonym substitution (WordNet or fastText).
   - Back-translation: FR → EN → FR using MarianMT or opus-MT.
2. Implement & Generate
   - For each of the 32 labeled examples, generate k augmented pseudo-examples (e.g. k=5).
   - Deduplicate and filter (e.g. reject if new text <50% overlap).
3. Merge & Re-split
   - Combine original 32 + synthetic N = 32×k examples.
   - Re-run the same CV split strategy, ensuring augmented copies of a given original stay in the same fold.
4. Re-train BERT
   - Exactly the same hyperparams as in (a).
   - Track performance uplift vs. the baseline.
5. Analysis
   - Compare metrics: ΔAccuracy, ΔF1.
   - Ablation: EDA vs. back-translation vs. combined.
   - Qualitative: inspect a few synthetic samples.

# 3. Zero-Shot Learning with LLM

# 4. Data Generation with LLM

# 5. Optimal Technique Application